In [1]:
import os
import numpy as np
import tifffile
import napari

from scipy.io import loadmat
from skimage import transform

from waveorder import waveorder_microscopy, fluorescence_microscopy, wavelet_softThreshold
from waveorder.io import WaveorderReader

# Table of contents
- View dataset
- Load data
    - Load raw images
    - Load calibration data
    - Load background images
- Recostruct fluorescence anisotropy channels
    - Register images
        - Crop edges
        - View registered images
    - Reconstruct Stokes images
        - Initialize Stokes reconstructor
        - Initialize fluorescence reconstructor
        - Denoise intensity images
            - View denoised intensity images
        - Reconstruct Raw Stokes images
        - Normalize Stokes images
            - View normalized Stokes images and background images
        - Correct background
            - View background corrected Stokes images
    - Compute anisotropy and orientation
    - Compute deconvolved fluorescence intensity
- View results

# View dataset

In [2]:
# assume data is in ~/Downloads folder
data_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
data_path = os.path.join(data_dir, 'miPolScope_fig1_u2os_labelfree.zarr')

# Check that data path exists
if not os.path.exists(data_path):
    raise ValueError('Data path does not exist.')

In [3]:
viewer = napari.Viewer()
layers = viewer.open(os.path.join(data_path, 'Row_0/Col_0/Pos_000'), plugin='napari-ome-zarr')

version mismatch: detected:FormatV01, requested:FormatV03
version mismatch: detected:FormatV03, requested:FormatV01


# Load data

## Load raw images

In [4]:
wo_data = WaveorderReader(data_path, data_type='zarr')
I = wo_data.get_zarr(0)
n_timepoints, n_channels, n_slices, *img_size = I.shape

In [5]:
print(
    f'Number of time points: {n_timepoints}\n'
    f'Number of channels: {n_channels}\n'
    f'Number of slices: {n_slices}\n'
    f'Image size: {img_size}'
)

Number of time points: 41
Number of channels: 8
Number of slices: 40
Image size: [640, 640]


As demonstration, we will analyze only the first 5 timepoints

In [6]:
n_timepoints = 5

In [7]:
# load data into memory
I = np.array(I[:n_timepoints, 4:]) # fluorescence data is in the last four channels

## Load calibration data

In [8]:
cal_data = loadmat(os.path.join(data_path,'calibration_fluor.mat'))

In [9]:
A = np.transpose(cal_data['A'].astype('float'), (2, 3, 0, 1)) # A has shape (size_Y, size_X, N_channels, N_Stokes)
black_level = cal_data['black_level'][0][0].astype('uint16')

tform0 = transform.AffineTransform(cal_data['tform0'].T)
tform45 = transform.AffineTransform(cal_data['tform45'].T)
tform90 = transform.AffineTransform(cal_data['tform90'].T)
tform135 = transform.AffineTransform(cal_data['tform135'].T)

## Load background images

In [10]:
S1_bg = cal_data['S1_bg'][0][0]
S2_bg = cal_data['S2_bg'][0][0]

# Reconstruct fluorescence anisotropy channels

## Register images

In [11]:
I_registered = np.zeros((n_timepoints, 4, n_slices, *img_size), dtype='float')

for t in range(n_timepoints):
    for c, tform in enumerate((tform0, tform45, tform90, tform135)):
        for z in range(n_slices):
            I_registered[t,c,z] = transform.warp(I[t,c,z], tform.inverse, preserve_range=True)

### Crop edges

In [12]:
I_registered = I_registered[..., 20:-20, 20:-20]
img_size = I_registered.shape[-2:]

### View registered images

In [13]:
viewer = napari.view_image(I_registered, contrast_limits=(99, 150))

## Reconstruct Stokes images

### Initialize Stokes reconstructor

In [14]:
# z projection parameters
z_chunk_size = 4
n_slices = n_slices//z_chunk_size

wavelength = 670 # in nm
NA_obj = 1.2 # Numerical Aperture of Objective
NA_illu = 0.4 # Numerical Aperture of Condenser
n_objective_media = 1.33 # refractive index of objective immersion media
mag = 30 # effective magnification
n_slices = n_slices # number of slices in z-stack
z_step_um = 0.25 * z_chunk_size # z-step size in um
pad_z = 5 # slices to pad for phase reconstruction boundary artifacts
pixel_size_um = 3.45 # camera pixel size in um
bg_correction = 'None' # BG correction method: 'None', 'local_fit', 'global'
mode = '3D' # phase reconstruction mode, '2D' or '3D'
use_gpu = False
gpu_id = 0 

In [15]:
z_defocus = -(np.r_[:n_slices] - n_slices // 2) * z_step_um # assumes stack starts from the bottom
swing = 0
ps = pixel_size_um / mag

reconstructor = waveorder_microscopy(img_dim=img_size,
                                     lambda_illu=wavelength/1000,
                                     ps=ps,
                                     NA_obj=NA_obj,
                                     NA_illu=NA_illu,
                                     z_defocus=z_defocus,
                                     chi=swing,
                                     n_media=n_objective_media,
                                     cali=True,
                                     bg_option=bg_correction,
                                     A_matrix=A,
                                     QLIPP_birefringence_only=True,
                                     pad_z=pad_z,
                                     phase_deconv=mode,
                                     illu_mode='BF',
                                     use_gpu=use_gpu,
                                     gpu_id=gpu_id)

### Initialize fluorescence reconstructor

In [16]:
lambda_emiss = [0.670]  # emission wavelength of the fluorescence channel (list, in um)

fluor_reconstructor = fluorescence_microscopy((*img_size, n_slices), lambda_emiss, ps, z_step_um, NA_obj, 
                                                n_media=n_objective_media,
                                                deconv_mode='3D-WF', 
                                                pad_z=3, 
                                                use_gpu=use_gpu, 
                                                gpu_id=gpu_id)

### Denoise intensity images

In [17]:
# z projection
I_denoised = np.reshape(I_registered, (n_timepoints, 4, n_slices, z_chunk_size, *img_size)).mean(axis=3)

# wavelet denoising
for c in range(4):
    I_denoised[:,c] = wavelet_softThreshold(I_denoised[:,c], 'db8', 1, level=2, axes=(1,2,3))

C:\Users\ivan.ivanov\AppData\Local\Continuum\miniconda3\envs\recorder\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 2 is too high: all coefficients will experience boundary effects.
  warnings.warn(


#### View denoised intensity images

In [18]:
viewer = napari.view_image(I_denoised, contrast_limits=(99, 150))

### Reconstruct Raw Stokes images

In [19]:
S_raw = np.zeros((n_timepoints, 3, n_slices, *img_size), dtype='float')

for t in range(n_timepoints):
    S_raw_ = reconstructor.Stokes_recon(np.moveaxis(I_denoised[t], 1, -1) - black_level)
    S_raw[t] = np.moveaxis(S_raw_, -1, 1)

### Normalize Stokes images

In [20]:
S_norm = np.zeros_like(S_raw)

for t in range(n_timepoints):
    for z in range(n_slices):
        S_norm[t,:,z] = reconstructor.Stokes_transform(S_raw[t,:,z])

#### View normalized Stokes images and background images

In [21]:
viewer = napari.Viewer()
viewer.add_image(S_norm[:,0], name='S0_norm', colormap='gray', contrast_limits=(0, 120))
viewer.add_image(S_norm[:,1], name='S1_norm', colormap='RdBu', visible=False, contrast_limits=(-0.25, 0.25))
viewer.add_image(S_norm[:,2], name='S2_norm', colormap='RdBu', visible=False, contrast_limits=(-0.25, 0.25))

viewer.add_image(S1_bg*np.ones(img_size), name='S1_bg', colormap='RdBu', visible=False, contrast_limits=(-0.25, 0.25))
viewer.add_image(S2_bg*np.ones(img_size), name='S2_bg', colormap='RdBu', visible=False, contrast_limits=(-0.25, 0.25))

<Image layer 'S2_bg' at 0x156c1ef0d00>

### Correct background

In [22]:
S_corr = np.zeros_like(S_norm)

S_corr[:,0] = S_norm[:,0]
S_corr[:,1] = S_norm[:,1] - S1_bg
S_corr[:,2] = S_norm[:,2] - S2_bg

#### View background corrected Stokes images

In [23]:
viewer = napari.Viewer()
viewer.add_image(S_corr[:,0], name='S0', colormap='gray', contrast_limits=(0, 120))
viewer.add_image(S_corr[:,1], name='S1', colormap='RdBu', visible=False, contrast_limits=(-0.25, 0.25))
viewer.add_image(S_corr[:,2], name='S2', colormap='RdBu', visible=False, contrast_limits=(-0.25, 0.25))

<Image layer 'S2' at 0x156c2dfbe50>

## Compute anisotropy and orientation

In [24]:
anisotropy, orientation = fluor_reconstructor.Fluor_anisotropy_recon(S_corr[:,1], S_corr[:,2])

## Compute deconvolved fluorescence intensity

In [25]:
blackground_level = [8]
I_deconvolved = np.zeros((n_timepoints, n_slices, *img_size), dtype='float')

for t in range(n_timepoints):
    I_ = fluor_reconstructor.deconvolve_fluor_3D(np.moveaxis(S_raw[t,0], 0, -1), blackground_level, reg=[1e-2], verbose=False)
    I_deconvolved[t] = np.moveaxis(I_, -1, 0)

# View results

In [26]:
viewer = napari.Viewer()
viewer.add_image(I_deconvolved, name='deconvolved intensity', colormap='gray', contrast_limits=(0, 200))
viewer.add_image(anisotropy, name='anisotropy', colormap='gray', visible=False, contrast_limits=(0, 0.2))
viewer.add_image(orientation, name='orientation', colormap='gray', visible=False, contrast_limits=(0, np.pi))

<Image layer 'orientation' at 0x156fe71bd00>